In [ ]:
import pandas as pd
import npc_lims
from npc_sessions import DynamicRoutingSession
from dynamic_routing_analysis import spike_utils, data_utils
import os
import pickle

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
ephys_sessions=tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, is_annotated=True,))

In [ ]:
except_dict={}
overwrite_files=False

for session_info in ephys_sessions[:]:
    
    try:
        trials=[]
        units=[]
        save_path=r"\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\new_annotations\single unit metrics"
        ##skip if file already exists
        if overwrite_files==False and os.path.isfile(os.path.join(save_path,session_info.id+'_stim_context_modulation.pkl')):
            print('session',session_info.id,'already processed; skipping')
            continue
        lick_save_path=os.path.join(save_path,'lick_modulation')
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        try:
            session=DynamicRoutingSession(session_info.id)
            trials=data_utils.load_trials_or_units(session, 'trials')
            units=data_utils.load_trials_or_units(session, 'units')

        except Exception as e:
            print('failed to load trials or units, skipping session')
            except_dict[session_info.id]=e
            continue

        spike_utils.compute_stim_context_modulation(trials, units, session_info, save_path)

        spike_utils.compute_lick_modulation(trials, units, session_info, lick_save_path)

        print(session_info.id,'done')

    except Exception as e:
        print(session_info.id,'failed')
        except_dict[session_info.id]=e


In [ ]:
except_dict